In [2]:

import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

In [3]:
# Using an already pretrained SBERT to compute cosine similarities which would then be used 

df=pd.read_csv("stsb_train.csv")
df=df.drop("idx",axis=1)
df_cosine_labels = df.copy() 
df_cosine_labels["label"] = df_cosine_labels["label"] /df_cosine_labels["label"].abs().max() 


df=pd.read_csv("stsb_validation.csv")
df=df.drop("idx",axis=1)
validation_cosine_labels = df.copy() 
validation_cosine_labels["label"] = validation_cosine_labels["label"] /validation_cosine_labels["label"].abs().max() 



In [4]:
train_examples = []
for i,row in df_cosine_labels.iterrows():
    train_examples.append(InputExample(texts=[row["sentence1"], row["sentence2"]],label=row["label"]))
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

val_examples = []
for i,row in validation_cosine_labels.iterrows():
    val_examples.append(InputExample(texts=[row["sentence1"], row["sentence2"]],label=row["label"]))


val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=16)

In [92]:
torch.save(model.state_dict(), "model_v2_lr1.pth")
lrs={"lr1":0.00001}

In [93]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    device = torch.device("cpu")
    

else:
    device = torch.device("mps")

In [5]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation

model = SentenceTransformer("distilbert-base-nli-mean-tokens")
train_loss = losses.CosineSimilarityLoss(model)
sentences1 = []
sentences2 = []
scores = []
for i,row in validation_cosine_labels.iterrows():
    sentences1.append(row["sentence1"])
    sentences2.append(row["sentence2"])
    scores.append(row["label"])

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=100,
    warmup_steps=100,
    evaluator=evaluator,
    evaluation_steps=500,
)

Epoch: 100%|██████████| 100/100 [1:50:35<00:00, 66.36s/it]


In [10]:
torch.save(model.state_dict(), "model_senbert.pth")
lrs={"lr1":0.00001}

In [8]:

df=pd.read_csv("stsb_test.csv")
df=df.drop("idx",axis=1)
test_cosine_labels = df.copy() 
test_cosine_labels["label"] = test_cosine_labels["label"] /test_cosine_labels["label"].abs().max() 
test_examples = []
for i,row in test_cosine_labels.iterrows():
    test_examples.append(InputExample(texts=[row["sentence1"], row["sentence2"]],label=row["label"]))


test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=1)


In [4]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("distilbert-base-nli-mean-tokens")
model.load_state_dict(torch.load("model_senbert.pth"))

# Two lists of sentences
sentences1 = [
"A person who cobbles .",
]

sentences2 = [
"Nonsense .",
]

# Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        sentences1[i], sentences2[i], cosine_scores[i][i]
    ))

A person who cobbles . 		 Nonsense . 		 Score: 0.4383


In [1]:
import numpy as np
data_points = np.random.rand(3387, 7929)

# Assuming 'centroid' is your centroid array of shape (1, 7929)
centroid = np.random.rand(1, 7929)

# Compute the Euclidean distance
distances = np.sqrt(np.sum((data_points - centroid) ** 2, axis=1))

In [6]:
data_points

array([[0.20668231, 0.45125576, 0.40175181, ..., 0.63991649, 0.21395359,
        0.88787819],
       [0.30406374, 0.16359059, 0.64448195, ..., 0.33988998, 0.19543703,
        0.906416  ],
       [0.69443658, 0.4957568 , 0.32649517, ..., 0.86524676, 0.25989218,
        0.76663841],
       ...,
       [0.74186748, 0.43780957, 0.23715375, ..., 0.45474528, 0.91249342,
        0.23555313],
       [0.01294029, 0.34696805, 0.99877482, ..., 0.99478881, 0.6308167 ,
        0.45844663],
       [0.40135945, 0.08331242, 0.57972066, ..., 0.33972442, 0.1514921 ,
        0.70476375]])

In [3]:
centroid.shape

(1, 7929)

In [5]:
distances[:,0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed